In [1]:
from Models.GAE_LSTM import GAE_LSTM_encoder, GAE_LSTM_decoder, GAE_LSTM_seq2seq
from torchinfo import summary
import torch
import scipy
import Models.Get_data as Gd
import pickle

import numpy as np
import os

In [2]:
import random
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(66)  # 你可以选择任意一个数字作为种子

In [3]:
directory = '/home/wl4023/data/Sibo_22Mar2024'
folders = [os.path.join(directory, f, 'hessian_') for f in os.listdir(directory) if f.startswith('case_')]

xyfile = "/home/wl4023/data/Sibo_22Mar2024/case_0/hessian_/xy_coords.npy"
pos = torch.tensor(np.load(xyfile), dtype=torch.float32)

sparse_graph = scipy.sparse.load_npz('/home/wl4023/data/Sibo_22Mar2024/new_sparse_matrix.npz')
indices = np.array(sparse_graph.nonzero())
values = sparse_graph.data
shape = sparse_graph.shape

# sparse edge tensor
edge_tensor = torch.sparse_coo_tensor(indices, values, shape, dtype=torch.float).coalesce()
edge_index = edge_tensor.indices()
edge_weight = edge_tensor.values()
edge_weight = edge_weight.unsqueeze(1)
print(edge_tensor.shape)
print(edge_index)
print(edge_weight)

print(f'{edge_weight.max():.8f}, {edge_weight.min():.8f}')

torch.Size([97149, 97149])
tensor([[    0,     0,     1,  ..., 97148, 97148, 97148],
        [    1,     3,     0,  ..., 97145, 97146, 97147]])
tensor([[0.0032],
        [0.0024],
        [0.0032],
        ...,
        [0.0039],
        [0.0043],
        [0.0038]])
0.02281803, 0.00025737


In [4]:
latent_space = 20
window_size = 10
step_size = 3
# batch_size = 5

num_mp_layers = [2, 2, 2]
num_clusters = [1000, latent_space]
clusters = torch.load(f'/home/wl4023/github repos/IRP/result/Latent space {latent_space}/clusters.pt')
centroids = torch.load(f'/home/wl4023/github repos/IRP/result/Latent space {latent_space}/centroids.pt')

In [5]:
dataset, length = Gd.get_all_nodes(folders[:-1])
dataset = np.expand_dims(dataset[:, :, 0], axis=2)

dataset = torch.tensor(dataset, dtype=torch.float32)
dataset = Gd.segment_data_torch(dataset, length, window_size, step_size)

data = Gd.MyDataset(dataset[:, 0:5, :, :], dataset[:, -5:, :, :])
# train_set, val_set = random_split(data, [0.8, 0.2])

# train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(val_set, batch_size=batch_size)

In [6]:
with open('/home/wl4023/github repos/IRP/preprocessor.pkl', 'rb') as f:
    preprocessor = pickle.load(f)

In [7]:
# Define the weights initialization function
def initialize_weights(m):
    if isinstance(m, torch.nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            torch.nn.init.constant_(m.bias, 0)

In [8]:
encoder = GAE_LSTM_encoder(latent_space, 1, num_mp_layers, clusters, centroids, 16, 3, 100)
encoder.apply(initialize_weights)
summary(encoder, input_data=(preprocessor.transform(data[0][0]), edge_index, edge_weight, pos))

Layer (type:depth-idx)                             Output Shape              Param #
GAE_LSTM_encoder                                   [16, 100]                 --
├─Encoder: 1-1                                     [20, 16]                  --
│    └─ModuleList: 2-261                           --                        (recursive)
│    │    └─Linear: 3-1                            [97149, 16]               32
│    └─ELU: 2-2                                    [97149, 16]               --
│    └─ModuleList: 2-261                           --                        (recursive)
│    │    └─Linear: 3-2                            [97149, 16]               272
│    └─ELU: 2-4                                    [97149, 16]               --
│    └─ModuleList: 2-261                           --                        (recursive)
│    │    └─Linear: 3-3                            [97149, 16]               272
│    └─ELU: 2-6                                    [97149, 16]               --
│    └

In [9]:
h_t, c_t, hidden_edge_index, hidden_edge_attr, edge_indices, edge_attrs, edge_indices_f2c, position, node_attrs, clusters = encoder(preprocessor.transform(data[0][0]), edge_index, edge_weight, pos)
decoder = GAE_LSTM_decoder(hidden_size=100,
                         latent_space=latent_space,
                         output_node_channel=1,
                         num_mp_layers=num_mp_layers,
                         hidden_channels=16,
                         n_mlp_mp=3)
decoder.apply(initialize_weights)
summary(decoder, input_data=(h_t, c_t, 7, hidden_edge_index, hidden_edge_attr, edge_indices, edge_attrs, edge_indices_f2c, position, node_attrs, clusters))

Layer (type:depth-idx)                             Output Shape              Param #
GAE_LSTM_decoder                                   [7, 97149, 1]             8,040
├─ELU: 1-1                                         [16, 20]                  --
├─ELU: 1-2                                         [16, 20]                  --
├─LSTM_cell: 1-3                                   [16, 20]                  9,680
├─Decoder: 1-4                                     [97149, 1]                --
│    └─ModuleList: 2-443                           --                        (recursive)
│    │    └─ModuleList: 3-195                      --                        (recursive)
│    └─ELU: 2-2                                    [90, 16]                  --
│    └─ModuleList: 2-443                           --                        (recursive)
│    │    └─ModuleList: 3-195                      --                        (recursive)
│    └─ELU: 2-4                                    [90, 16]              

In [11]:
model = GAE_LSTM_seq2seq(latent_space=latent_space,
                         hidden_size=100,
                         input_node_channel=1,
                         output_node_channel=1,
                         num_mp_layers=num_mp_layers,
                         clusters=clusters,
                         centroids=centroids,
                         hidden_channels=16,
                         n_mlp_mp=3)
model.apply(initialize_weights)

summary(model, input_data=(preprocessor.transform(data[0][0]), 7, edge_index, edge_weight, pos))

Layer (type:depth-idx)                                  Output Shape              Param #
GAE_LSTM_seq2seq                                        [7, 97149, 1]             --
├─GAE_LSTM_encoder: 1-1                                 [16, 100]                 --
│    └─Encoder: 2-1                                     [20, 16]                  --
│    │    └─ModuleList: 3-261                           --                        (recursive)
│    │    └─ELU: 3-2                                    [97149, 16]               --
│    │    └─ModuleList: 3-261                           --                        (recursive)
│    │    └─ELU: 3-4                                    [97149, 16]               --
│    │    └─ModuleList: 3-261                           --                        (recursive)
│    │    └─ELU: 3-6                                    [97149, 16]               --
│    │    └─ModuleList: 3-301                           --                        (recursive)
│    │    └─ELU: 3-8    